In [4]:
import cv2
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.models.segmentation import deeplabv3_resnet101
import numpy as np

def load_model():
    model = deeplabv3_resnet101(pretrained=True)
    model.eval()
    return model

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image).unsqueeze(0)

def segment_image(model, image_tensor):
    with torch.no_grad():
        output = model(image_tensor)['out']
        _, predicted = torch.max(output, 1)
        return predicted.squeeze().cpu().numpy()

def draw_bounding_boxes(image, bounding_boxes, segmentation):
    for bbox in bounding_boxes:
        x, y, w, h = bbox
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cropped_segmentation = segmentation[y:y+h, x:x+w]
        cv2.imshow("Segmentation", cropped_segmentation)
        cv2.waitKey(0)

def main():
    image_path = "image_examples/prueba.jpg"
    bounding_boxes = np.array([[15, 400, 269, 525], [300, 40, 435, 395]])

    model = load_model()
    image_tensor = preprocess_image(image_path)
    segmentation = segment_image(model, image_tensor)

    image = cv2.imread(image_path)
    draw_bounding_boxes(image, bounding_boxes, segmentation)

    cv2.imshow("Image with Bounding Boxes", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

    


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
